In [16]:
import tensorflow as tf
import numpy as np
import math

# Comment
# This is solution to problem of figuring out falling body problem written with the use of ChatGPT.
# I prompted the problem and proposed the machine learning approach, chose some hyperparameters, asked to generate data 
# and finally asked to calculate confidence intervals and importance of each input variable to the output.
# All code with other comments are generated. 
# Dataset comes as vectors (D, W, R) where D is distance of fall, W is weight of ball and R is it's radius. 
# We try to model time of fall, given this data. Note that equation of this phenomenon is given as: math.sqrt(2 * d / 9.81) which depends on D only.
# End of Comment

# Define the neural network model
model = tf.keras.Sequential([ tf.keras.layers.Dense(10, activation='relu', input_shape=(3,)), tf.keras.layers.Dense(1)])

# Compile the model
model.compile(optimizer='sgd', loss='mse')

# Generate training dataset
X_train = np.random.rand(300, 3)
y_train = np.zeros(300)

for i in range(300):
    d = X_train[i][0] * 100
    t = math.sqrt(2 * d / 9.81)
    y_train[i] = t
    
# Generate evaluation dataset
X_eval = np.random.rand(100, 3)
y_eval = np.zeros(100)

for i in range(100):
    d = X_eval[i][0] * 100
    t = math.sqrt(2 * d / 9.81)
    y_eval[i] = t


# calculate mean and standard deviation of training data
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

# normalize training data
X_train_norm = (X_train - mean) / std

# normalize evaluation data using the mean and std of training data
X_eval_norm = (X_eval - mean) / std



In [17]:
# Train the model for 100 epochs
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
10/10 [==============================] - 1s 3ms/step - loss: 4.7494
Epoch 2/100
10/10 [==============================] - 0s 4ms/step - loss: 0.9064
Epoch 3/100
10/10 [==============================] - 0s 4ms/step - loss: 0.5604
Epoch 4/100
10/10 [==============================] - 0s 4ms/step - loss: 0.5060
Epoch 5/100
10/10 [==============================] - 0s 3ms/step - loss: 0.4644
Epoch 6/100
10/10 [==============================] - 0s 3ms/step - loss: 0.4229
Epoch 7/100
10/10 [==============================] - 0s 4ms/step - loss: 0.3874
Epoch 8/100
10/10 [==============================] - 0s 3ms/step - loss: 0.3518
Epoch 9/100
10/10 [==============================] - 0s 5ms/step - loss: 0.3205
Epoch 10/100
10/10 [==============================] - 0s 7ms/step - loss: 0.2905
Epoch 11/100
10/10 [==============================] - 0s 6ms/step - loss: 0.2643
Epoch 12/100
10/10 [==============================] - 0s 7ms/step - loss: 0.2375
Epoch 13/100
10/10 [=================

In [18]:
scores = model.evaluate(X_eval, y_eval)
print("Mean squared error on evaluation set: ", scores)

4/4 [==============================] - 0s 4ms/step - loss: 0.0523
Mean squared error on evaluation set:  0.05232156813144684


In [19]:
import math
# obtain predicted outputs on the evaluation set
y_pred = model.predict(X_eval)

scores = np.zeros((len(y_pred)))
for i in range(len(y_pred)):
    scores[i] = math.pow(y_eval[i] - y_pred[i], 2)

scores

4/4 [==============================] - 0s 3ms/step


array([3.58286458e-02, 3.03733272e-02, 3.58878667e-03, 2.97682934e-01,
       2.37588442e-02, 1.38190527e-03, 3.65865219e-03, 2.89031064e-02,
       3.32334037e-03, 4.38236749e-02, 6.67461468e-04, 2.80585417e-01,
       1.20513844e+00, 6.48770418e-02, 2.03490589e-02, 2.74202508e-03,
       5.20723845e-03, 2.78252698e-02, 3.75328359e-02, 2.75605544e-02,
       2.68158366e-04, 2.00185381e-02, 8.40656872e-04, 2.41606587e-02,
       5.86328452e-02, 7.65669252e-02, 1.58086419e-02, 2.49517940e-02,
       3.55300655e-02, 1.30450427e-02, 1.42794336e-02, 4.58968062e-01,
       1.83598499e-02, 2.62536377e-02, 4.34924960e-02, 7.08817876e-03,
       3.34026755e-05, 1.92427852e-02, 6.94749124e-03, 3.24835911e-02,
       6.84515350e-02, 3.76628339e-02, 8.86549208e-03, 3.40822200e-02,
       1.74397311e-02, 3.46873645e-02, 1.07127206e-02, 1.55458264e-02,
       2.49749231e-02, 4.66785536e-03, 1.58321526e-02, 2.65280898e-02,
       9.35655333e-03, 4.10639917e-03, 4.70572270e-03, 2.26910481e-02,
      

In [20]:
import numpy as np
from scipy import stats

# calculate the standard deviation of the sample
sd = np.sqrt(np.var(scores))

# calculate the standard error of the mean
sem = sd / np.sqrt(len(scores))

# calculate the t-value for the 95% confidence level with n-1 degrees of freedom
t_value = stats.t.ppf(0.975, len(scores)-1)

# calculate the confidence interval
ci = sem * t_value

# print the result
print("95% confidence interval: [{:.4f}, {:.4f}]".format(scores.mean()-ci, scores.mean()+ci))


95% confidence interval: [0.0248, 0.0798]


In [24]:
# get weights of the hidden layer
hidden_layer_weights = model.layers[0].get_weights()[0]

# calculate input variable importance
input_importance = []
for i in range(X_train.shape[1]):
    importance = 0
    for j in range(hidden_layer_weights.shape[1]):
        importance += X_train[:, i] * hidden_layer_weights[i, j] * model.layers[1].get_weights()[0][j, 0]
    input_importance.append(abs(importance).sum())

# normalize input variable importance
input_importance /= sum(input_importance)

print(input_importance)

[0.90836207 0.07348952 0.01814841]
